# Devoir 1 : La classification de deux façons avec les ngrams

Bienvenue à votre premier devoir! Quand vous le soumetterez sur ZoneCours, assurez-vous de renommer le fichier: **"Devoir1_{prénom}_{nomdefamille}.ipynb"**. Les instructions sont en français, par contre comme beaucoup des matériaux du cours sont en anglais, vous trouverez la traduction anglaise avec chaque problème. Votre devoir est due au plus tard à 11:59 PM le 20 février, 2025. Si vous avez travaillé avec un ou des collègues, écrivez leur nom ici:

In [ ]:
__autheur__ = "{Leroy Tiojip}"
__collaborateurs__ = "{Leur nom, si il y a plusieurs personnes séparez les par un point-virgule; sinon, à laissez vide}"

Toutes les librairies python dont vous aurez besoin sont ici. Si une librairie qui semble être utile est manquante, c'est par exprès pour que vous écriviez vos fonctions par vous même. Il ne faut pas modifier cette case, ni rajouter de libraires ailleurs. Par contre, vous pouvez utiliser toutes fonctions que vous trouvez utiles dans ces librairies.

In [5]:
from nltk.corpus import movie_reviews
import string
import re
import random
import math
from collections import Counter
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
random.seed(202401)
torch.manual_seed(202401)
device = torch.device('cpu')

## Problème 1 [18 points]
Dans ce problème, vous allez implémenter un classificateur Naive Bayes de critiques de films qui utilise des n-grams comme variables pour prédire si une critique est positive ou négative. Vous comparerez trois classificateurs différents qui utiliseront respectivement des  unigrams, bigrams et trigrams.

*In this problem you will be implementing a movie review Naive Bayes classifier that uses n-gram features to predict whether a review was positive or negative. You will compare three different classifiers which will respectively use unigram features, bigram features and trigram features.* 

La première étape consiste à télécharger les données et à créer notre répartition test/train. 

In [8]:
review_classes = movie_reviews.categories()
pos_reviews = [(movie_reviews.raw(fileid), 'pos') for fileid in movie_reviews.fileids(categories=['pos'])]
neg_reviews = [(movie_reviews.raw(fileid), 'neg') for fileid in movie_reviews.fileids(categories=['neg'])]
train_dataset = pos_reviews[:800] + neg_reviews[:800]
test_dataset = pos_reviews[800:] + neg_reviews[800:]

1.  **[3 points]** Vous allez maintenant devoir écrire une fonction de prétraitement de text qui prend une critique de film, supprime la ponctuation, met tous les mots en minuscules et les divise en tokens en fonction de l'espacement. Assurez-vous de ne pas avoir de tokens vides.

    *You will now have to write a preprocessing function which takes a review, removes punctuation, lower cases all the words and splits them into tokens based on  any type of spacing. Make sure you do not have empty tokens.*

In [10]:
def preprocess(review:str):
    tokenized_review = []
    punctuations = list(string.punctuation)
    ## TO DO
    review = review.lower()
    review = review.translate(str.maketrans('', '', string.punctuation.replace("'", "")))
    tokenized_review =re.findall(r'\b\w+\b', review)
    
    ##
    return tokenized_review

2. **[3 points]** Afin d'extraire les n-grams représentatifs d'une classe, nous aurons besoin de quelques fonctions helpers. La première que vous devez implémenter est une fonction qui prend un texte tokenisé et un hyperparamètre *n* et retourne l'ensemble unique de ngrams présents dans le texte tokenisé, soit le vocabulaire. Notez que chaque ngram doit être de type tuple, et nous supposons qu'aucun padding n'est effectué.

   *In order to extract the n-gram features of a class, we will need a couple helper functions. The first one you will have to implement is a function which takes a tokenized text and a hyperparameter *n* and returns the unique set of ngrams present in the tokenized text. Note that each ngram should be of type tuple, and we assume that no padding is done.*

In [12]:
def get_ngram_vocabulary(tokenized_reviews:list[tuple[list[str], str]], n:int):
    ngram_vocab = []
    ## TO DO
    for tokens, label in tokenized_reviews: 
        if len(tokens) >= n:  
            ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)] 
            ngram_vocab.extend(ngrams)  


    
    ##
    return ngram_vocab

3. **[4 points]** Créez une fonction qui récupère les ngrams pour une classe particulière de textes dans un ensemble de données. Cette fonction doit d'abord (a) filtrer l'ensemble de textes tokenisés par la valeur du paramètre de classe, puis (b) extraire le modèle n-grams représentatif de la classe compte tenu du vocabulaire. Veillez à mettre en œuvre Laplace smoothing pour chaque modèle de ngrams. Une fois de plus, aucun padding n'est nécessaire.
   
   *Create a function that retrieves the ngram features for a particular class of texts in a dataset. This function should first (a) filter the tokenized texts dataset based on the class parameter, and then (b) extract the n-gram model representative of the class given the vocabulary. Make sure to implement Laplace smoothing for each ngram model. Once again, no padding is necessary.*

In [14]:
def get_class_ngram_features(tokenized_reviews:list[tuple[list[str], str]], review_class:str, ngram_vocab:list[tuple], vocab_size:int):
    class_ngrams = {}
    ## TO DO
    filtered_reviews = [tokens for tokens, label in tokenized_reviews if label == review_class]
    class_ngram_counts = Counter()
    for tokens in filtered_reviews:
        n = len(ngram_vocab[0])  # Taille des n-grammes 
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
        class_ngram_counts.update(ngrams)

    total_ngrams = sum(class_ngram_counts.values())  # Total des n-grammes de cette classe
    for ngram in ngram_vocab:
        class_ngrams[ngram] = (class_ngram_counts.get(ngram, 0) + 1) / (total_ngrams + vocab_size)

    
    ##
    return class_ngrams

#### Classificateur Naive Bayes avec Ngram

En utilisant toutes les fonctions helpers que vous venez d'implémenter, entraînons maintenant un classificateur NB en utilisant des unigram, bigram et trigram. 
Voici une fonction qui renvoie la probabilité logarithmique à priori d'une classe dans un ensemble de données, quel que soit le contenu de la classe.

*Using all of the helper functions you just implemented, let's now train a NB classifier using unigram, bigram, and trigram features. 
Here is a helper function that returns the prior log likelihood of a particular class in a dataset irrespective of class content.*

In [16]:
def get_prior(tokenized_reviews:list[tuple[list[str], str]], review_class:str):
    prob = 0.0
    total = len(tokenized_reviews)
    class_count = 0
    for _ , c in tokenized_reviews:
        if c == review_class:
            class_count+=1
    prob = class_count/total
    return math.log(prob)

4. **[4 points]** Écrivez une fonction supplémentaire qui, étant donné un critique de film test, retourne la classe la plus probable en fonction de la distribution à priori sur les classes et de la distribution multinomiale de ngrams par classes. Notez que cette fonction doit ignorer tous les ngrams qui ne font pas partie du modèle de ngrams entraîné.

   *Write one more helper function which given a test review returns its most likely review class conditioned on the prior distribution over classes and some ngram multinomial distribution over classes. Note, it should ignore any ngrams that are not part of the trained ngram model features.*

In [18]:
def get_class_prediction(review:list[str], ngram_multinomial_dist:dict[str, dict[str,float]], class_prior_dist:dict[str, float], n:int):
    prediction = ""
    ## TO DO
    test_ngrams = [tuple(review[i:i+n]) for i in range(len(review) - n + 1)]
    class_scores = {} 
    for review_class, prior_log_prob in class_prior_dist.items():
        log_prob = prior_log_prob  
        for ngram in test_ngrams:
            if ngram in ngram_multinomial_dist[review_class]:  
                log_prob += math.log(ngram_multinomial_dist[review_class][ngram])

        class_scores[review_class] = log_prob

    prediction = max(class_scores, key=class_scores.get)

    
    ##
    return prediction

Et voici notre classificateur.

In [20]:
def NB_ngram_classifier(train_dataset:list[tuple[str, str]], test_dataset:list[tuple[str, str]], review_classes:list[str]):
    # 1: We preprocess both the test and train datasets
    tokenized_test = [(preprocess(review), review_class) for review, review_class in  test_dataset]
    tokenized_train = [(preprocess(review), review_class) for review, review_class in  train_dataset]
    
    # 2: Let's now extract the unigram, bigram, and trigram vocabularies we will use.
    unigram_vocab = get_ngram_vocabulary(tokenized_train, 1)
    bigram_vocab = get_ngram_vocabulary(tokenized_train, 2)
    trigram_vocab = get_ngram_vocabulary(tokenized_train, 3)
    
    # 3: We must now *learn* the set of features for each category in the tokenized train data. We represent our features as a multinomial distributions over review_classes. 
    # We will compare three distributions, (a) unigram, (b) bigram, and (c) trigram multinomials. Additionally, we get the prior distribution over review classes.
    unigram_multinomial_dist = dict()
    bigram_multinomial_dist = dict()
    trigram_multinomial_dist = dict()
    class_prior_dist = dict()
    for review_class in review_classes:
        # n-gram multinomial
        unigram_multinomial_dist[review_class] = get_class_ngram_features(tokenized_train, review_class, unigram_vocab, len(unigram_vocab))
        bigram_multinomial_dist[review_class] = get_class_ngram_features(tokenized_train, review_class, bigram_vocab, len(unigram_vocab))
        trigram_multinomial_dist[review_class] = get_class_ngram_features(tokenized_train, review_class, trigram_vocab, len(unigram_vocab))
        # class prior
        class_prior_dist[review_class] = get_prior(tokenized_train, review_class)
        
    # 4: Now that we have our class features by ngram model we will try to predict the class of each review in our tokenized test dataset. 
    results = []
    for index, (tokenized_review, class_label) in enumerate(tokenized_test):
        results.append({'review_id': index, 
                            'class_label':class_label, 
                            'unigram_prediction': get_class_prediction(tokenized_review, unigram_multinomial_dist, class_prior_dist, 1),
                            'bigram_prediction': get_class_prediction(tokenized_review, bigram_multinomial_dist, class_prior_dist, 2),
                            'trigram_prediction': get_class_prediction(tokenized_review, trigram_multinomial_dist, class_prior_dist, 3),
                           })
    return results

In [21]:
# This should run for a minute or two.
results = NB_ngram_classifier(train_dataset, test_dataset, review_classes)

#### Résultats

Comparons maintenant les performances relatives de nos classificateurs NB avec unigram, bigram et trigram.

5. **[4 points]** Écrivez une fonction qui retourne les scores moyens de précision, recall et F1, ainsi que le accuracy score moyen pour chaque classificateur.

   *Write a function that returns the overall precision, recall, and balanced F1 scores across all classes as well as overall accuracy for each classifiers predictions.*

In [24]:
def get_accuracies(results:dict):
    accuracies = {'unigram':dict([('precision', 0.0),('recall', 0.0), ('f1', 0.0), ('accuracy', 0.0)]),
                  'bigram':dict([('precision', 0.0),('recall', 0.0), ('f1', 0.0), ('accuracy', 0.0)]),
                  'trigram':dict([('precision', 0.0),('recall', 0.0), ('f1', 0.0), ('accuracy', 0.0)])}
    ## TO DO
    metrics = {
        'unigram': {'TP': 0, 'FP': 0, 'FN': 0},
        'bigram': {'TP': 0, 'FP': 0, 'FN': 0},
        'trigram': {'TP': 0, 'FP': 0, 'FN': 0}
    }

    for res in results:
        true_label = res['class_label']
        
        for model in ['unigram', 'bigram', 'trigram']:
            pred_label = res[f'{model}_prediction']
            
            if pred_label == true_label:
                metrics[model]['TP'] += 1  
            else:
                metrics[model]['FP'] += 1  
                metrics[model]['FN'] += 1  

    for model in ['unigram', 'bigram', 'trigram']:
        TP = metrics[model]['TP']
        FP = metrics[model]['FP']
        FN = metrics[model]['FN']
        total = TP + FP + FN

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        accuracy = TP / total if total > 0 else 0.0

        accuracies[model]['precision'] = precision
        accuracies[model]['recall'] = recall
        accuracies[model]['f1'] = f1
        accuracies[model]['accuracy'] = accuracy

    
    ##
    return accuracies

In [25]:
accuracies = get_accuracies(results)

In [26]:
pd.DataFrame.from_dict(accuracies, orient='index')

,precision,recall,f1,accuracy
unigram,0.6800,0.6800,0.6800,0.515152
bigram,0.7975,0.7975,0.7975,0.663202
trigram,0.8250,0.8250,0.8250,0.702128


## Problème 2 [18 points]

Nous allons maintenant essayer une autre forme de classificateur, la régression logistique. Vous allez mettre en œuvre un un simple réseau neuronal qui implémente la régression logistique et qui utilise des ngrams comme variables indépendantes, pour prédire la classe de critiques de films. Nous utiliserons les mêmes ensembles de données test et train que pour le problème 1. 

*We will now extend problem 1 and consider another form of classifier, logistic regression. You will implement a logistic regression classifier via a simple single layered neural network that uses ngrams as its features, or independent variables, to predict the class of reviews. We will use the same test and train datasets as problem 1.*

1. **[3 points]** Écrivez une fonction qui retourne un vocabulaire de ngram pour n'importe quel *n*. Il est important que la taille du vocabulaire soit limitée à la taille maximale spécifiée. Il doit inclure les *max_vocab_size* ngrams les plus fréquents dans *tokenized_reviews*.

   *Write a helper function that returns an ngram_vocabulary for any *n*. Importantly the vocabulary size should be limited to the maximum vocabulary size specified. It should include the *max_vocab_size* most frequent ngrams across all reviews.*

In [29]:
def get_ngram_vocabulary_maxsize(tokenized_reviews:list[tuple[list[str], str]], n:int, max_vocab_size:int):
    ngram_vocab = []
    ## TO DO
    all_ngrams = []
    for tokens, _ in tokenized_reviews:
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
        all_ngrams.extend(ngrams) 
        
    ngram_counts = Counter(all_ngrams)
    most_common_ngrams = [ngram for ngram, _ in ngram_counts.most_common(max_vocab_size)]
    ngram_vocab = most_common_ngrams 
    
    
    ##
    return ngram_vocab

2. **[3 points]** Écrivez une fonction qui prend un vocabulaire de ngrams et une critique de film tokenisé et retourne une représentation vectorisée des comptes de ngrams dans la critique. Le type du vecteur retourné doit être un torch tensor de floats.

   *Write a helper function which takes an ngram vocabulary and a tokenized review and returns a vectorized representation of the ngram counts in the review. The return type should be a torch tensor of floats.*

In [31]:
def get_vectorized_review(review:list[str], ngram_vocab:list[str]):
    vectorized_review = torch.Tensor([0.0])
    ## TO DO
    n = len(ngram_vocab[0]) if ngram_vocab else 1 
    review_ngrams = [tuple(review[i:i+n]) for i in range(len(review) - n + 1)]
    vector = [review_ngrams.count(ngram) for ngram in ngram_vocab]
    vectorized_review = torch.tensor(vector, dtype=torch.float32)
    
    
    ##
    return vectorized_review

3. **[4 points]** Écrivez une classe python ReviewDataset qui implémente un objet torch Dataset. Elle doit prendre en input *tokenized_reviews* (une liste de tuples (tokenized_review, review_class)) ainsi qu'un vocabulaire de ngrams. Votre fonction d'initialisation doit vectoriser toutes les critiques, de sorte que self.vectorized_reviews soit un torch tensor, ou une matrice de taille |tokenized_reviews| x |ngram_vocab|. Quant à self.labels, il s'agit également d'un torch tensor de taille |tokenized_reviews| x |review_classes|, où chaque index est un vecteur one-hot tel que [1,0] est 'neg' et [0,1] est 'pos'.
  
   *Write a ReviewDataset class implements a torch Dataset object. It should take as input tokenized_reviews (a list of (tokenized_review, review_class) tuples) as well as a vocabulary of ngrams. Your initializing function should vectorize all reviews, such that self.vectorized_reviews is a torch tensor, or matrix of size |tokenized_reviews| x |ngram_vocab|. As for your self.labels, this should also be a torch tensor of size |tokenized_reviews| x |review_classes|, were each index is a one hot vector such that [1,0] is'neg' and [0,1] is 'pos' class labels.*

In [33]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_reviews:list[tuple[list[str], str]], ngram_vocab:list[tuple]):
        vectorized_reviews = [] 
        labels = [] 
        ## TO DO
        class_map = {'neg': [1, 0], 'pos': [0, 1]} 
        for tokens, label in tokenized_reviews:
            vectorized_review = get_vectorized_review(tokens, ngram_vocab)
            vectorized_reviews.append(vectorized_review)
            labels.append(class_map[label][0])

        
        ##
        self.vectorized_reviews = torch.stack(vectorized_reviews, dim=0)
        self.labels = F.one_hot(torch.Tensor(labels).long()).float()
        self.length = len(labels)

    def __getitem__(self, index):
        ## TO DO
        vectorized_review = self.vectorized_reviews[index]  
        label = self.labels[index] 

        
        ##
        return vectorized_review, label, index

    def __len__(self):
        return self.length


4. **[2 points]** Écrivez une fonction get_dataloader qui prend en input un ensemble de données tokenisé, un vocabulaire de ngrams, la taille des batchs et un hyperparamètre shuffle. Elle doit utiliser la classe ReviewDataset que vous avez définie précédemment et retourner un objet de type torch.utils.data.DataLoader.

   *Write a get_dataloader function which takes as input a tokenized dataset, an ngram vocabulary, a batch size, and a shuffle hyperparameter. It should use the ReviewDataset class you previously defined and return an object of type torch.utils.data.DataLoader.*

In [35]:
def get_dataloader(tokenized_reviews:list[tuple[list[str], str]], ngram_vocab:list[tuple], batch_size:int, shuffle:bool):
    ## TO DO
    dataset = ReviewDataset(tokenized_reviews, ngram_vocab)  
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    
    ##
    return dataloader

#### Régression logistique avec un réseau neuronal simple

Voici notre classe modèle LR que nous utiliserons comme base pour nos classificateurs LR ngrams.


In [37]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

5. **[4 points]** Complétez la fonction d'apprentissage pour notre classificateur LR ngram. Votre code doit s'assurer d'entraîner le modèle pour le nombre d'epochs spécifié sur toutes les batchs du dataloader.
  
   *Complete the training function for our LR ngram classifier. Your code should make sure to train the model for the number of epochs specified over all batches in the training dataloader.*

In [39]:
def train_LR_ngram_classifier(train_dataset:list[tuple[str, str]], review_classes:list[str], n:int): 
    lr = 0.001
    batch_size = 32
    epochs = 5

    tokenized_train = [(preprocess(review), review_class) for review, review_class in  train_dataset]
    ngram_vocab = get_ngram_vocabulary_maxsize(tokenized_train, n, 30000)

    train_dataloader = get_dataloader(tokenized_train, ngram_vocab, batch_size, shuffle=True)

    model = LogisticRegression(len(ngram_vocab), len(review_classes)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    ## TO DO
    loss_function = nn.CrossEntropyLoss() 

    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for batch in train_dataloader:
            x_batch, y_batch, _ = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            predictions = model(x_batch)

            y_batch = torch.argmax(y_batch, dim=1)  
            loss = loss_function(predictions, y_batch)  
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            
            predicted_labels = torch.argmax(predictions, dim=1)
            correct_predictions += (predicted_labels == y_batch).sum().item()
            total_samples += y_batch.size(0)

        accuracy = correct_predictions / total_samples * 100
    
            
    ## 
    return ngram_vocab, model   

In [40]:
# This should run for a couple minutes.
unigram_vocab, unigram_model = train_LR_ngram_classifier(train_dataset, review_classes, 1)
bigram_vocab, bigram_model = train_LR_ngram_classifier(train_dataset, review_classes, 2)
trigram_vocab, trigram_model = train_LR_ngram_classifier(train_dataset, review_classes, 3)

#### Évaluation de nos classificateurs LR 
Voici la fonction d'évaluation de nos classificateurs. Elle prend les données test ainsi qu'un classificateur LR ngrams et son vocabulaire correspondant et renvoie les prédictions du modèle pour chaque item test.

*Here is the evaluation function for our classifiers. It takes the test data as well as some trained LR ngram classifier and its corresponding vocabulary and returns the model predictions for each item in the test data.*

In [42]:
def eval_LR_ngram_classifier(test_dataset:list[tuple[str, str]], ngram_vocab:list[tuple], model):
    batch_size = 32
    
    tokenized_test = [(preprocess(review), review_class) for review, review_class in  test_dataset]
    
    test_dataloader = get_dataloader(tokenized_test, ngram_vocab, batch_size, shuffle=False)

    indexes = []
    labels = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for batch_vectorized_reviews, batch_labels, batch_indexes in test_dataloader:
            x = batch_vectorized_reviews.to(device)
            y_pred = model(x)

            indexes += batch_indexes.tolist()
            labels += batch_labels.tolist()
            predictions += y_pred.tolist()
    
    results = [{'review_id':index, 'class_label':label, 'prediction': pred} for index, label, pred in zip(indexes, labels, predictions)]
    return results

In [43]:
unigram_results = eval_LR_ngram_classifier(test_dataset, unigram_vocab, unigram_model)
bigram_results = eval_LR_ngram_classifier(test_dataset, bigram_vocab, bigram_model)
trigram_results = eval_LR_ngram_classifier(test_dataset, trigram_vocab, trigram_model)

#### Résultats

Comparons maintenant les performances relatives de nos classificateurs LR unigram, bigram et trigram.

6. **[2 points]** Écrivez une fonction qui retourne les scores moyens de précision, recall et F1 pour toutes les classes, ainsi que le accuracy score d'un classificateur. Notez que les étiquettes et prédictions sont d'un type différent que dans le problème 1, assurez-vous de les traiter correctement.
  
   *Write a function that returns the overall precision, recall, and balanced F1 scores across all classes as well as overall accuracy of a classifier. Note, that labels and predictions are a different type than in problem 1, make sure to handle them correctly.*

In [46]:
def get_accuracy_scores(results:list[dict]):
    accuracy = dict([('precision', 0.0),('recall', 0.0), ('f1', 0.0), ('accuracy', 0.0)])
    ## TO DO
    true_positive = Counter()
    false_positive = Counter()
    false_negative = Counter()
    total_correct = 0
    total = len(results)
    
    for res in results:
        true_label = res['class_label'] if isinstance(res['class_label'], int) else torch.argmax(torch.tensor(res['class_label'])).item()
        predicted_label = res['prediction'] if isinstance(res['prediction'], int) else torch.argmax(torch.tensor(res['prediction'])).item()
        
        if true_label == predicted_label:
            total_correct += 1
            true_positive[true_label] += 1
        else:
            false_positive[predicted_label] += 1
            false_negative[true_label] += 1
    
    classes = set(true_positive.keys()).union(false_positive.keys()).union(false_negative.keys())
    precision = {cls: true_positive[cls] / (true_positive[cls] + false_positive[cls]) if (true_positive[cls] + false_positive[cls]) > 0 else 0.0 for cls in classes}
    recall = {cls: true_positive[cls] / (true_positive[cls] + false_negative[cls]) if (true_positive[cls] + false_negative[cls]) > 0 else 0.0 for cls in classes}
    f1 = {cls: 2 * (precision[cls] * recall[cls]) / (precision[cls] + recall[cls]) if (precision[cls] + recall[cls]) > 0 else 0.0 for cls in classes}
    
    accuracy['accuracy'] = total_correct / total if total > 0 else 0.0
    accuracy['precision'] = sum(precision.values()) / len(classes) if classes else 0.0
    accuracy['recall'] = sum(recall.values()) / len(classes) if classes else 0.0
    accuracy['f1'] = sum(f1.values()) / len(classes) if classes else 0.0

    
    ##
    return accuracy

In [47]:
accuracies = {'unigram':get_accuracy_scores(unigram_results),
                  'bigram':get_accuracy_scores(bigram_results),
                  'trigram':get_accuracy_scores(trigram_results)}

In [48]:
pd.DataFrame.from_dict(accuracies, orient='index')

,precision,recall,f1,accuracy
unigram,0.892510,0.8925,0.892499,0.8925
bigram,0.868246,0.8675,0.867433,0.8675
trigram,0.812381,0.8075,0.806745,0.8075


## QUESTION BONUS [2 points supplémentaires]

Comparez les résultats des six classificateurs et écrivez quelques phrases sur ce que vous observez. Comment le changement de l'hyperparamètre **n** affecte-t-il les performances des classificateurs NB par rapport aux classificateurs LR ? Remarquez-vous des différences entre les classements de accuracy et de F1 score, et pourquoi?

*Compare all six classifier results and write a couple sentences about what you observe. How does changing the **n** hyperparameter affect performance of NB versus LR classifiers? Do you notice differences between accuracy and F1 score rankings, why might that be?*

[ VOTRE RÉPONSE ICI ]: On remaque que pour les classificateurs LR ; une augmentation de l'hyperparamètre n entraîne une baisse de la performance (evite overfitting); là où à contrario , dans le cas des classficateurs NB , on assiste à une augmentation de la performance. cela peut s'expliquer car NB analyse chaque n-gramme séparément sans souffrir autant d’overfitting.

Dans NB ; l’accuracy et le F1-score augmentent de manière concordante avec donc le modèle gère mieux les classes avec plus de contexte.
Dans LR ;l’accuracy baisse légèrement avec le F1-score , donc les erreurs sont plus équilibrées entre faux positifs et faux négatifs.